Тестовое задание

Цель: необходимо обработать сырые данные, поступившие от парсера.

Описание: в файле `hotels_parsing_result.xlsx` два листа - `platform1`, `platform2` с данными о гостиницах от двух разных платформ в одном регионе.

Поля:
- `id` - Идентификатор в БД;
- `create_time` - Дата создания;
- `title` - Название;
- `hotel_type_original` - Тип гостиницы;
- `city` - Город;
- `address` - Адрес;
- `rating` - Рейтинг;
- `rating_5` - Рейтинг по пятибалльной шкале;
- `review_count` - Количество отзывов;
- `star_rating` - Звёздность;
- `rooms_count` - Количество номеров;
- `contact_social` - Контакты соц. сетей;
- `description` - Описание;
- `email` - адрес строкой, несколько значений через запятую;
- `phone` - номер телефона строкой, несколько значений через запятую;
- `website` - сайты строкой, несколько значений через запятую;
- `uid` - Уникальный идентификатор гостиницы на платформе, не может быть разным у одной гостиницы, и не может повторятся у разных гостиниц, но в рамках одной платформы;
- `parsing_time` - Время сбора;
- `lat` - Широта;
- `lon` - Долгота.

Состав данных:
Записи могут повторяться, и самые актуальные данные идут в конце.
Если какое-то поле по конкретной гостинице пустое в последней записи, но встречалось ранее, то необходимо его взять из более ранних записей.
Поля телефона, email, сайтов могут содержать лишние символы, несколько записей и прочее.
Формат записи названия, адреса, типа гостиницы отличается в разных платформах.

Задача:
1. Собрать от каждой платформы финальный список гостиниц, в котором по каждой гостинице внутри платформы будет только одна запись с самыми актуальными и полными данными.
2. Почистить данные.
3. Поля телефона, email, сайтов распарсить и сохранить как списки в одинаковом формате (address@domen.org, 79234553322, domen.ru)
4. Вывести топ 10 по каждой платформе, по параметрам: больше всего телефонов, больше всего отзывов.
5. Вывести квадрат координат размером 1км на 1км, где больше всего гостиниц.
6. Задача со *, объединить данные от двух платформ, по критерию, который надо придумать. Например: вывести все гостиницы, которые есть в платформе 1 и нет в платформе 2; или вывести топ-10 гостиниц которые есть в обоих платформах, по суммарному количеству отзывов.

Требования:
1. Результат должен быть представлен Jupyter notebook.
2. Результат должен воспроизводится автоматически с нуля при повторном запуске, ручные правки должны быть учтены в коде.
3. Комментарии в коде приветствуются.


Декомпозиция:
1) Собрать от каждой платформы финальный список гостиниц, в котором по каждой гостинице внутри платформы будет только одна запись с самыми актуальными и полными данными.
- Объединить данные гостиниц из обеих платформ.
- Для каждой гостиницы выбрать запись с наибольшим рейтингом.
- Для каждой гостиницы выбрать запись с наиболее полным набором данных.
- Удалить дубликаты гостиниц.

2) Почистить данные.
- Удалить ненужные символы и пробелы.
- Проверить данные на наличие ошибок и опечаток.
- Заменить пропущенные значения на None или на среднее/медиану.

3) Поля телефона, email, сайтов распарсить и сохранить как списки в одинаковом формате (address@domen.org, 79234553322, domen.ru).
- Разделить данные на отдельные столбцы.
- Распарсить данные каждого столбца.
- Сохранить распарсенные данные в соответствующие списки.

4) Вывести топ-10 по каждой платформе, по параметрам:<br>
a. больше всего телефонов,<br>
b. больше всего отзывов.
- Отсортировать гостиницы по количеству телефонов.
- Отсортировать гостиницы по количеству отзывов.
- Вывести первые 10 гостиниц из каждого списка.

5) Вывести квадрат координат размером 1 км на 1 км, где больше всего гостиниц.
- Определить координаты каждой гостиницы.
- Найти квадрат с наибольшим количеством гостиниц.
- Вывести координаты этого квадрата.

6) Задача со *, объединить данные от двух платформ, по критерию, который вы придумаете.
- Объединить данные гостиниц из обеих платформ.
- Применить критерий для объединения данных.

7) Вывести результаты в соответствии с заданием.


In [1]:
# Импорт библиотек
import pandas as pd
from fuzzywuzzy import process
import re
import numpy as np
from IPython.display import display
from itertools import permutations

In [ ]:
# Чтение данных из Excel файла
#df = pd.read_excel('hotels_parsing_result.xlsx')
#df.head()

In [2]:
# Чтение данных из Excel файла и объединение двух платформ
first_sheet = pd.read_excel('hotels_parsing_result.xlsx', sheet_name='platform1')  # Загрузка первого листа
second_sheet = pd.read_excel('hotels_parsing_result.xlsx', sheet_name='platfrom2') # Загрузка второго листа

# Объединение листов в один датафрейм
df = pd.concat([first_sheet, second_sheet], ignore_index=True)

df.head()

,id,create_time,title,hotel_type_original,city,address,rating,rating_5,review_count,region,...,rooms_count,contact_social,description,email,phone,website,uid,parsing_time,lat,lon
0,81842,2023-09-26 14:45:42.910962,Апартаменты Baltia Западный Пляж,Apartment,Зеленоградск,"улица Приморская, д. 31, Зеленоградск",10.0,5.00,0.0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,10595357,2023-09-26 20:56:02.013106,54.951103,20.456526
1,86341,2023-09-26 15:01:55.184815,Отель Авиатор,Hotel,Уфа,"улица Мушникова, д.28, Уфа",8.7,4.35,5.0,2,...,8.0,NaN,NaN,NaN,NaN,NaN,8332081,2023-09-26 20:56:02.013106,54.783913,56.116150
2,86420,2023-09-26 15:02:12.005085,Клеопатра,Hotel,Уфа,"Коммунистическая ул., д. 53, Уфа",6.3,3.15,10.0,2,...,15.0,NaN,NaN,NaN,NaN,NaN,7724907,2023-09-26 20:56:02.013106,54.726463,55.946445
3,86421,2023-09-26 15:02:12.225334,Татьяна,Hotel,Сибай,"Учалинская улица, 37, Сибай",10.0,5.00,2.0,2,...,15.0,NaN,NaN,NaN,NaN,NaN,6598197,2023-09-26 20:56:02.013106,52.719230,58.658596
4,85898,2023-09-26 15:00:19.217359,Мини-Отель Чемодан,Mini-hotel,Стерлитамак,"улица Дружбы, д.28 Д, Стерлитамак",8.0,4.00,0.0,2,...,23.0,NaN,NaN,NaN,NaN,NaN,8622634,2023-09-26 20:56:02.013106,53.642567,55.932686


In [3]:
# Функция получения полной информации о датафрейме
def data_info (data):
    print('\033[1m' + 'Первые 10 строк:' + '\033[0m')
    display(data.head(10))
    print('-------------')
    print('\033[1m' + 'Последние 10 строк:' + '\033[0m')
    display(data.tail(10))
    print('-------------')
    print('\033[1m' + 'Типы данных:' + '\033[0m')
    display(data.info(memory_usage='deep'))
    print('-------------')
    print('\033[1m' + 'Полное статистическое описание:' + '\033[0m')
    display(data.describe(include = "all"))
    print('-------------')
    print('\033[1m' + 'Категориальные признаки:' + '\033[0m')
    display(data.describe(include=[object]))
    print('-------------')
    print('\033[1m' + 'Пропуски:' + '\033[0m')
    # Обход всех столбцов и проверка наличия пропущенных значений
    for i in data.columns:
        if data[i].isna().sum() > 0:
            print(f"Столбец {i} имеет {data[i].isna().sum()} пропусков\n")
        else:
            print(f"В столбце {i} пропусков НЕТ\n")
    print('-------------')
    print('\033[1m' + 'Доля пропусков от всех данных:' + '\033[0m')
    display(data.isna().mean().sort_values(ascending=False)*100)
    print('-------------')
    print('\033[1m' + 'Явные дубликаты:' + '\033[0m')
    if data.duplicated().sum() > 0:
        print('Дубликатов: ', data.duplicated().sum())
    else:
        print('Явных дублей НЕТ')

data_info(df)

Первые 10 строк:


,id,create_time,title,hotel_type_original,city,address,rating,rating_5,review_count,region,...,rooms_count,contact_social,description,email,phone,website,uid,parsing_time,lat,lon
0,81842,2023-09-26 14:45:42.910962,Апартаменты Baltia Западный Пляж,Apartment,Зеленоградск,"улица Приморская, д. 31, Зеленоградск",10.0,5.00,0.0,2,...,NaN,NaN,NaN,NaN,NaN,NaN,10595357,2023-09-26 20:56:02.013106,54.951103,20.456526
1,86341,2023-09-26 15:01:55.184815,Отель Авиатор,Hotel,Уфа,"улица Мушникова, д.28, Уфа",8.7,4.35,5.0,2,...,8.0,NaN,NaN,NaN,NaN,NaN,8332081,2023-09-26 20:56:02.013106,54.783913,56.116150
2,86420,2023-09-26 15:02:12.005085,Клеопатра,Hotel,Уфа,"Коммунистическая ул., д. 53, Уфа",6.3,3.15,10.0,2,...,15.0,NaN,NaN,NaN,NaN,NaN,7724907,2023-09-26 20:56:02.013106,54.726463,55.946445
3,86421,2023-09-26 15:02:12.225334,Татьяна,Hotel,Сибай,"Учалинская улица, 37, Сибай",10.0,5.00,2.0,2,...,15.0,NaN,NaN,NaN,NaN,NaN,6598197,2023-09-26 20:56:02.013106,52.719230,58.658596
4,85898,2023-09-26 15:00:19.217359,Мини-Отель Чемодан,Mini-hotel,Стерлитамак,"улица Дружбы, д.28 Д, Стерлитамак",8.0,4.00,0.0,2,...,23.0,NaN,NaN,NaN,NaN,NaN,8622634,2023-09-26 20:56:02.013106,53.642567,55.932686
5,85899,2023-09-26 15:00:19.415692,Hilton Garden Inn Ufa Riverside,Hotel,Уфа,"улица Аксакова 4, Уфа",9.1,4.55,616.0,2,...,167.0,NaN,NaN,NaN,NaN,NaN,6342481,2023-09-26 20:56:02.013106,54.719320,55.931340
6,85900,2023-09-26 15:00:19.631849,Отель Hampton by Hilton Уфа,Hotel,Уфа,"улица 50 лет Октября, 17, Уфа",9.2,4.60,460.0,2,...,160.0,NaN,NaN,NaN,NaN,NaN,6790074,2023-09-26 20:56:02.013106,54.737415,55.969723
7,85901,2023-09-26 15:00:19.871941,Отель ВолнаСити,Hotel,Уфа,"улица Заки Валиди, д.64/2, Уфа",8.1,4.05,26.0,2,...,33.0,NaN,NaN,NaN,NaN,NaN,8468945,2023-09-26 20:56:02.013106,54.715140,55.955162
8,85902,2023-09-26 15:00:20.095657,AZIMUT Сити Отель Уфа,Hotel,Уфа,"пр-т Октября, д. 81, Уфа",8.7,4.35,988.0,2,...,190.0,NaN,NaN,NaN,NaN,NaN,7403650,2023-09-26 20:56:02.013106,54.774017,56.025784
9,85903,2023-09-26 15:00:20.316568,Отель Ural Taur,Hotel,Нефтекамск,"Юбилейный проспект, д.18, Нефтекамск",8.5,4.25,0.0,2,...,122.0,NaN,NaN,NaN,NaN,NaN,8745681,2023-09-26 20:56:02.013106,56.087997,54.236900


-------------
Последние 10 строк:


,id,create_time,title,hotel_type_original,city,address,rating,rating_5,review_count,region,...,rooms_count,contact_social,description,email,phone,website,uid,parsing_time,lat,lon
8279,49842,2023-09-19 21:46:09.669101,"Красный ключ, гостевой дом",hotel,ufa,"Россия, Республика Башкортостан, Нуримановский...",5.0,5.0,2.0,2,...,4.0,"[{""social"": ""vk"", ""contact"": ""https://vk.com/g...",NaN,NaN,"+73472119603, +79867047997",gostinitsakrasneykluch.taplink.ws,70000001061154148,2023-09-20 04:43:49.208656,55.382057,56.676784
8280,49844,2023-09-19 21:46:10.157066,"Урман, база отдыха",hotel_rest,NaN,"Россия, Республика Башкортостан, Караидельский...",NaN,NaN,NaN,2,...,NaN,NaN,NaN,NaN,+79196165583,www.urmanrb.ru,70000001069874874,2023-09-20 04:43:49.722438,55.520995,56.640452
8281,49845,2023-09-19 21:46:10.372936,Дом отдыха,hotel_rest,NaN,"Россия, Республика Башкортостан, Караидельский...",NaN,NaN,NaN,2,...,NaN,"[{""social"": ""vk"", ""contact"": ""https://vk.com/c...",NaN,NaN,+79869603777,Забронировать,70000001063063811,2023-09-20 04:43:49.973058,55.574140,56.700792
8282,49846,2023-09-19 21:46:10.617272,У капитана Флинта,hotel_rest,NaN,"Россия, Республика Башкортостан, Караидельский...",NaN,NaN,NaN,2,...,NaN,"[{""social"": ""vk"", ""contact"": ""https://vk.com/u...",NaN,NaN,+79050061962,Забронировать,70000001069853112,2023-09-20 04:43:50.223143,55.795121,56.932888
8283,49847,2023-09-19 21:46:10.836733,"Любимое место, база отдыха",hotel_rest,NaN,"Россия, Республика Башкортостан, Караидельский...",NaN,NaN,NaN,2,...,NaN,"[{""social"": ""vk"", ""contact"": ""https://vk.com/g...",NaN,NaN,"+79236157271, +79178094499",Забронировать,70000001077384780,2023-09-20 04:43:50.459270,55.947259,57.090599
8284,49848,2023-09-19 21:46:11.061452,"Караидель, физкультурно-оздоровительный комплекс",hotel_rest,NaN,"Россия, Республика Башкортостан, Караидельский...",NaN,NaN,NaN,2,...,NaN,"[{""social"": ""vk"", ""contact"": ""https://vk.com/c...",NaN,NaN,+73472827979,www.kara-idel.ru,70000001069863555,2023-09-20 04:43:50.769707,55.794670,56.864993
8285,49851,2023-09-19 21:46:11.764997,"Lounge villa, база отдыха",hotel_rest,NaN,"Россия, Республика Башкортостан, Туймазинский ...",NaN,NaN,NaN,2,...,NaN,NaN,NaN,NaN,+79236157271,Забронировать,70000001069856457,2023-09-20 04:43:51.527665,54.434405,54.018789
8286,49852,2023-09-19 21:46:11.981114,"Лена, база отдыха",hotel_rest,NaN,"Россия, Республика Башкортостан, Илишевский ра...",NaN,NaN,NaN,2,...,NaN,NaN,NaN,NaN,+79236157271,Забронировать,70000001069870275,2023-09-20 04:43:51.802655,55.288257,54.483841
8287,49853,2023-09-19 21:46:12.205246,База отдыха,hotel_rest,NaN,"Россия, Республика Башкортостан, Краснокамский...",NaN,NaN,NaN,2,...,NaN,"[{""social"": ""vk"", ""contact"": ""https://vk.com/s...",NaN,NaN,"+73478331555, +79236157271",www.bazasauzovo.ru,70000001069866157,2023-09-20 04:43:52.091311,55.950944,53.791774
8288,49854,2023-09-19 21:46:12.424735,Гостиница Ашкадар,NaN,sterlitamak,"Россия, Республика Башкортостан, Стерлитамак г...",NaN,NaN,NaN,2,...,NaN,NaN,NaN,NaN,NaN,NaN,7600610350202940,2023-09-20 04:43:52.417084,53.638674,55.937977


-------------
Типы данных:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8289 entries, 0 to 8288
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   8289 non-null   int64  
 1   create_time          8289 non-null   object 
 2   title                8289 non-null   object 
 3   hotel_type_original  8269 non-null   object 
 4   city                 7144 non-null   object 
 5   address              8289 non-null   object 
 6   rating               7641 non-null   float64
 7   rating_5             7641 non-null   float64
 8   review_count         7641 non-null   float64
 9   region               8289 non-null   int64  
 10  star_rating          902 non-null    float64
 11  rooms_count          2518 non-null   float64
 12  contact_social       506 non-null    object 
 13  description          197 non-null    object 
 14  email                67 non-null     object 
 15  phone      

...

-------------
Категориальные признаки:


,create_time,title,hotel_type_original,city,address,contact_social,description,email,phone,website,parsing_time
count,8289,8289,8269,7144,8289,506,197,67,1076,734,8289
unique,8289,5733,19,139,5012,239,31,30,722,236,1352
top,2023-09-26 14:45:42.910962,Гостиница,Apartment,Калининград,"Россия, Республика Башкортостан, Гафурийский р...","[{""social"": ""vk"", ""contact"": ""https://vk.com/b...",<br />•&nbsp;Единый кол-центр 24/7 подберёт дл...,hostelufa@richotels.ru,+79872540575,Забронировать,2023-09-26 20:56:02.013106
freq,1,65,4069,2360,30,8,15,4,13,151,5522


-------------
Пропуски:
В столбце id пропусков НЕТ

В столбце create_time пропусков НЕТ

В столбце title пропусков НЕТ

Столбец hotel_type_original имеет 20 пропусков

Столбец city имеет 1145 пропусков

В столбце address пропусков НЕТ

Столбец rating имеет 648 пропусков

Столбец rating_5 имеет 648 пропусков

Столбец review_count имеет 648 пропусков

В столбце region пропусков НЕТ

Столбец star_rating имеет 7387 пропусков

Столбец rooms_count имеет 5771 пропусков

Столбец contact_social имеет 7783 пропусков

Столбец description имеет 8092 пропусков

Столбец email имеет 8222 пропусков

Столбец phone имеет 7213 пропусков

Столбец website имеет 7555 пропусков

В столбце uid пропусков НЕТ

В столбце parsing_time пропусков НЕТ

Столбец lat имеет 573 пропусков

Столбец lon имеет 573 пропусков

-------------
Доля пропусков от всех данных:


email                  99.191700
description            97.623356
contact_social         93.895524
website                91.144891
star_rating            89.118108
phone                  87.018941
rooms_count            69.622391
city                   13.813488
rating                  7.817590
rating_5                7.817590
review_count            7.817590
lat                     6.912776
lon                     6.912776
hotel_type_original     0.241284
create_time             0.000000
region                  0.000000
address                 0.000000
uid                     0.000000
parsing_time            0.000000
title                   0.000000
id                      0.000000
dtype: float64

-------------
Явные дубликаты:
Явных дублей НЕТ


## Картина данных
1) Очень много пропусков в полях `email`, `description`, `contact_social`, `website`, `star_rating`, `phone` и `rooms_count` - удалять нельзя, заменить нечем. Заменим пропуски (NaN) в числовых полях (`rooms_count`, `review_count`, `rating` и `rating_5`, и др.) на 0. Пропусков в поле `hotel_type_original` всего 20, это некритично, может быть они отсеются на этапе чистки данных.

2) В типах данных есть явные проблемы (например, датафрейм занимает в памяти почти 7 МВ, что избыточно). Значит надо преобразовать их так:
- `id` в uint16;
- `create_time` в DateTime64;
- `title` в object;
- `hotel_type_original` в object;
- `city` в object;
- `address` в object;
- `rating` в uint8;
- `rating_5` в uint8;
- `review_count` в uint16;
- `region` в uint8;
- `star_rating` в float64;
- `rooms_count` в uint16;
- `contact_social` в object;
- `description` в object;
- `email` в object;
- `phone` в object;
- `website` в object;
- `uid` в uint64;
- `parsing_time` в DateTime64;
- `lat` в float32;
- `lon` в float32.

3) То, что функция не нашла явные дубликаты, не говорит об их отсутствии в неявном виде. Значит нужно будет их поискать и исследовать.

In [10]:
# Заменяем пропущенные значения на 0
df['rating'] = df['rating'].fillna(0)
df['rating_5'] = df['rating_5'].fillna(0)
df['rooms_count'] = df['rooms_count'].fillna(0)
df['review_count'] = df['review_count'].fillna(0)
df['lat'] = df['lat'].fillna(0)
df['lon'] = df['lon'].fillna(0)

In [11]:
# Преобразование типов данных
df['id'] = df['id'].astype('uint16')
df['create_time'] = pd.to_datetime(df['create_time'])
df['rating'] = df['rating'].astype('uint8')
df['rating_5'] = df['rating_5'].astype('uint8')
df['review_count'] = df['review_count'].astype('uint16')
df['region'] = df['region'].astype('uint8')
df['rooms_count'] = df['rooms_count'].astype('uint16')
df['uid'] = df['uid'].astype('uint64')
df['parsing_time'] = pd.to_datetime(df['parsing_time'])
df['lat'] = df['lat'].astype('float32')
df['lon'] = df['lon'].astype('float32')
display(df.info(memory_usage='deep'))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8289 entries, 0 to 8288
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   id                   8289 non-null   uint16        
 1   create_time          8289 non-null   datetime64[ns]
 2   title                8289 non-null   object        
 3   hotel_type_original  8269 non-null   object        
 4   city                 7144 non-null   object        
 5   address              8289 non-null   object        
 6   rating               8289 non-null   uint8         
 7   rating_5             8289 non-null   uint8         
 8   review_count         8289 non-null   uint16        
 9   region               8289 non-null   uint8         
 10  star_rating          902 non-null    float64       
 11  rooms_count          8289 non-null   uint16        
 12  contact_social       506 non-null    object        
 13  description          197 non-null

None

In [41]:
# Напишем функцию проверки неявных дубликатов по сочетанию полей
descriptions = {
    'id': 'Идентификатор в БД',
    'title': 'Название гостиницы',
    'address': 'Адрес расположения',
    'uid': 'Уникальный идентификатор',
    'lat': 'Широта',
    'lon': 'Долгота',
}

def check_duplicates(df, fields, descriptions):
    descriptions_dict = dict(zip(fields, descriptions))
    descriptions_used = [descriptions_dict.get(field, 'Описание не найдено') for field in fields]
    
    dup_count = df.duplicated(fields).sum()
    
    if dup_count != 0:
        combination = 'Сочетание: ' + ', '.join(descriptions_used)
        print(combination)
        print(f"Количество дубликатов: {dup_count}\n")

# Получаем все возможные длины комбинаций полей из словаря
for length in range(1, len(descriptions)+1):
    # Получаем все комбинации полей данной длины
    for combination_fields in permutations(descriptions, length):
        combination_values = [descriptions[field] for field in combination_fields]
        check_duplicates(df, combination_fields, combination_values)

Сочетание: Идентификатор в БД
Количество дубликатов: 215

Сочетание: Название гостиницы
Количество дубликатов: 2556

Сочетание: Адрес расположения
Количество дубликатов: 3277

Сочетание: Уникальный идентификатор
Количество дубликатов: 2218

Сочетание: Широта
Количество дубликатов: 4133

Сочетание: Долгота
Количество дубликатов: 4041

Сочетание: Название гостиницы, Адрес расположения
Количество дубликатов: 2241

Сочетание: Название гостиницы, Уникальный идентификатор
Количество дубликатов: 2218

Сочетание: Название гостиницы, Широта
Количество дубликатов: 1792

Сочетание: Название гостиницы, Долгота
Количество дубликатов: 1792

Сочетание: Адрес расположения, Название гостиницы
Количество дубликатов: 2241

Сочетание: Адрес расположения, Уникальный идентификатор
Количество дубликатов: 2218

Сочетание: Адрес расположения, Широта
Количество дубликатов: 2558

Сочетание: Адрес расположения, Долгота
Количество дубликатов: 2557

Сочетание: Уникальный идентификатор, Название гостиницы
Количество

Мы имеем 1675 повторов - дубликатов по сочетанию: `Название гостиницы`, `Уникальный идентификатор`, `Адрес расположения`, `Долгота`, `Широта`. В тоже время по сочетанию: `Уникальный идентификатор`, `Адрес расположения`, `Название гостиницы` мы получили 2218 дубликатов. Ещё мы имеем 2547 дубликатов в сочетании: `Адрес расположения`, `Широта`, `Долгота`, что говорит о нескольких гостиницах по одному адресу, или о другом написания одного названия. Интересное сочетание `Широта`, `Долгота`, где 3873 дубликатов, то есть плотность гостиниц на территориях довольно высокая.

Одновременно, по одному полю более интересная картина. По `Идентификатору в БД` количество дубликатов составило 215, видимо есть повторы в одной платформе. По `Названию гостиницы` имеем 2556 повторов, надо разбираться с этим. `Адрес расположения` показал 3277 дубликатов, что ещё интереснее. Ну и поле `Уникальный идентификатор` выдал 2218 дублей.

In [15]:
# Функция для выбора записи с наибольшим рейтингом среди одинаковых по полю 'title'
def select_highest_rating(records):
    # Сортируем записи по убыванию рейтинга
    sorted_records = sorted(records, key=lambda x: x['rating'], reverse=True)
    
    # Возвращаем первую запись в отсортированном списке, так как она имеет наибольший рейтинг
    return sorted_records[0]

# Вызываем функцию и получаем запись с наибольшим рейтингом
highest_rating_record = select_highest_rating(df)
print(highest_rating_record)  # Выведет запись с наибольшим рейтингом

TypeError: string indices must be integers, not 'str'

In [13]:
def select_highest_rating(records):
    unique_titles = set(record['title'] for record in records)
    highest_rating_records = []
    
    for title in unique_titles:
        records_with_title = [record for record in records if record['title'] == title]
        highest_rating_record = max(records_with_title, key=lambda x: x['rating'])
        highest_rating_records.append(highest_rating_record)
    
    return highest_rating_records

highest_rating_records = select_highest_rating(df)
for record in highest_rating_records:
    print(record) # Выведет запись с наибольшим рейтингом

TypeError: string indices must be integers, not 'str'

In [17]:
# Группировка данных по uid и выбор последнего значения для каждой гостиницы
grouped_df = df.groupby('uid').tail(1)

grouped_df.head()

,id,create_time,title,hotel_type_original,city,address,rating,rating_5,review_count,region,...,rooms_count,contact_social,description,email,phone,website,uid,parsing_time,lat,lon
0,16306,2023-09-26 14:45:42.910962,Апартаменты Baltia Западный Пляж,Apartment,Зеленоградск,"улица Приморская, д. 31, Зеленоградск",10,5,0,2,...,0,NaN,NaN,NaN,NaN,NaN,10595357,2023-09-26 20:56:02.013106,54.951103,20.456526
1,20805,2023-09-26 15:01:55.184815,Отель Авиатор,Hotel,Уфа,"улица Мушникова, д.28, Уфа",8,4,5,2,...,8,NaN,NaN,NaN,NaN,NaN,8332081,2023-09-26 20:56:02.013106,54.783913,56.116150
2,20884,2023-09-26 15:02:12.005085,Клеопатра,Hotel,Уфа,"Коммунистическая ул., д. 53, Уфа",6,3,10,2,...,15,NaN,NaN,NaN,NaN,NaN,7724907,2023-09-26 20:56:02.013106,54.726463,55.946445
3,20885,2023-09-26 15:02:12.225334,Татьяна,Hotel,Сибай,"Учалинская улица, 37, Сибай",10,5,2,2,...,15,NaN,NaN,NaN,NaN,NaN,6598197,2023-09-26 20:56:02.013106,52.719231,58.658596
4,20362,2023-09-26 15:00:19.217359,Мини-Отель Чемодан,Mini-hotel,Стерлитамак,"улица Дружбы, д.28 Д, Стерлитамак",8,4,0,2,...,23,NaN,NaN,NaN,NaN,NaN,8622634,2023-09-26 20:56:02.013106,53.642567,55.932686


In [ ]:
# Функция для выбора записи с наибольшим рейтингом
def get_highest_rating(hotels):
    # Здесь должна быть логика для выбора записи с наибольшим рейтингом
    # Например, можно взять запись с наибольшим значением рейтинга
    return max(hotels.items(), key=lambda item: item[1]['rating'])

In [ ]:
# Функция для выбора наиболее полной записи
def get_most_complete_record(hotels):
    # Сначала находим запись с наибольшим значением create_time
    most_recent_record = max(hotels.items(), key=lambda item: item[1]['create_time'])
    return most_recent_record

In [ ]:
# Выбираем наиболее полную запись для каждой гостиницы
most_complete_hotels = {hotel_id: get_most_complete_record(hotels)[1] for hotel_id, hotels in grouped_df.items()}

# Выбираем запись с наибольшим рейтингом для каждой гостиницы
highest_rating_hotels = {hotel_id: get_highest_rating(hotels)[1] for hotel_id, hotels in grouped_df.items()}

# Объединяем результаты и удаляем дубликаты
final_hotels = {
    **most_complete_hotels,
    **highest_rating_hotels
}

# Удаляем дубликаты
final_hotels = {k: v for k, v in final_hotels.items() if v not in final_hotels.values()}

# Выводим результат
print(final_hotels)

In [ ]:
# Чистка данных
def standardize_address(address):
# Преобразование адреса в стандартный формат
    address = address.lower() # Приведение адреса к нижнему регистру
    address = address.replace(" ", "") # Удаление пробелов
    address = address.replace(",", "") # Удаление запятых
    address = address.replace(".", "") # Удаление точек
    return address

def clean_data(dataframe):
    # Чистка полей телефона, email, сайтов
    dataframe['phone'] = dataframe['phone'].apply(lambda x: ','.join(filter(None, re.split(r'\D+', str(x)))))
    dataframe['email'] = dataframe['email'].apply(lambda x: process.extractOne(x, dataframe['email'].tolist())[0] if x in dataframe['email'].tolist() else x)
    dataframe['website'] = dataframe['website'].apply(lambda x: x.lower().strip().replace('https://', '').replace('http://', '').split('/')[0])

    return dataframe

In [ ]:
cleaned_dataframe = clean_data(grouped_df)
# Вывод результатов
display(cleaned_dataframe)

In [ ]:
# Распределение данных по спискам
# Можно использовать регулярные выражения для распарсивания строк
# Функция для распарсивания строк
def parse_strings(series):
    # Регулярное выражение для поиска контактных данных
    pattern = r'(\S+)'  # (\S+) для версии Python >= 3.7
    results = []
    for item in series:
        matches = re.findall(pattern, item)
        results.extend(matches)
    return results

# Распределение данных по спискам
df['phone'] = parse_strings(df['phone'])
df['email'] = parse_strings(df['email'])
df['website'] = parse_strings(df['website'])

print(df)

In [ ]:
# Вывод топа 10 гостиниц по количеству телефонов и отзывов
top_phones = grouped_data.sort_values(['phone', 'platform'], ascending=[False, False]).head(10)
top_reviews = grouped_data.sort_values(['review_count', 'platform'], ascending=[False, False]).head(10)

In [ ]:
# Вычисление квадрата координат размером 1 км на 1 км, где больше всего гостиниц
# Это может потребовать дополнительной логики для определения границ квадрата

In [ ]:
# Задача со звездочкой
# Здесь можно реализовать различные сценарии объединения данных, например:
# merged_data = pd.merge(df1, df2, on='uid', how='left').fillna(method='ffill')
# top_merged = merged_data.sort_values(['review_count', 'platform'], ascending=[False, False]).head(10)

In [ ]:
# Вывод результатов
display(grouped_data)
display(top_phones)
display(top_reviews)